In [1]:
import pandas as pd
import numpy as np
import re

In [6]:
df_map = pd.read_csv('24_regions.csv', index_col = 0)

In [2]:
df = pd.read_excel('minosvita_df.xlsx')

In [184]:
"""
РЕГУЛЯРНІ ВИРАЗИ ТА ФІЛЬТРИ
"""

reg_number = '(№\s*\d+)' #витягує число перед яким є символ №
reg_quotes = """["'«](.*?)["'»]{1}?""" #витягує текст в лапках трьох видів
lot_regex = '(((\DЗОШ\D)|(\Dзош\D)|(\DСШ\D)|(\DДНЗ\D)|(\DДНЗ\D)).(\d+))' #витягує номер поруч з ключовими словами
reg_city = "(м\.\w+)" # витягує назву міста після "м."

#набір ключових слів-фільтрів
filter_shkola = ['школ', 'шкіль',  'загальноосвітн',
                 'гімназ',  'ліце[йя]', 'колегіум'] # 'виховн' прибрав

filter_shkola_name = 'Школа'

filter_sadik = ['днз', 'садок', 'дошкільн', ' садка', 'ясла', 'ясел']
filter_sadik_name = 'ДНЗ'

filter_sport = ['юнацька спортивна', 'дюсш', 'дитячо-спортивна']
filter_sport_name = 'ДЮСШ'

filter_music = ['музич', "дмш"]
filter_music_name = 'Музична школа'

filter_art = ['мистецтв', 'дхш']
filter_art_name = 'Школа мистецтв'

filter_hud = ['худож']
filter_hud_name = 'Художня школа'

filter_poza = ['позашкільн', 'міжшкільн', 'МСВЗШ', 'спеціальна вечірня']
filter_poza_name = 'Позашкільне'

filter_inshe = 'Інше'

In [11]:
# Створення таблиці з головним органом і відповідним містом

regions = ['Суми', 'Київ', 'Полтава', 'Харків', 'Чернігів', 'Вінниця', 'Львів', 'Миколаїв', 'Дніпро',
           'Чернівці', 'Запоріжжя', 'Одеса', 'Черкаси', 'Херсон', 'Рівне', 'Тернопіль', 'Ужгород', 
           'Луцьк', 'Івано-Франківськ', 'Житомир', 'Кропивницький', 'Хмельницький']

gol_organ = ['Сумська область',
 'м.Київ',
 'Полтавська область',
 'Харківська область',
 'Чернігівська область',
 'Вінницька область',
 'Львівська область',
 'Миколаївська область',
 'Дніпровська область',
 'Чернівецька область',
 'Запорізька область',
 'Одеська область',
 'Черкаська область',
 'Херсонська область',
 'Рівненська область',
 'Тернопільська область',
 'Ужгородська область',
 'Луцька область',
 'Івано-Франківська область',
 'Житомирська область',
 'Кропивницька область',
 'Хмельницька область']

city_organ = pd.DataFrame(list(gol_organ), columns=['gol_organ'])
city_organ['city'] = pd.Series(regions)
city_organ = city_organ[['gol_organ', 'city']]

# city_organ.sample()

In [12]:
"""
ФУНКЦІЯ, ЯКА ВИТЯГУЄ ІМЕНА ШКІЛ І ПРИВОДИТЬ ДО ВІДПОВІДНОСТІ ФОРМАТУ ЗАПИСУ В ТАБЛИЦІ МАП

"""

def fix_table():    
    
    df_fixed = df.copy()
    df_fixed['type'] = pd.np.where(df_fixed["name"].str.contains('|'.join(filter_sport), case = False), 
                                   filter_sport_name,
                            pd.np.where(df_fixed["name"].str.contains('|'.join(filter_music), case = False), 
                                        filter_music_name,
                            pd.np.where(df_fixed["name"].str.contains('|'.join(filter_art), case = False), 
                                        filter_art_name,
                            pd.np.where(df_fixed["name"].str.contains('|'.join(filter_hud), case = False), 
                                        filter_hud_name,
                            pd.np.where(df_fixed["name"].str.contains('|'.join(filter_poza), case = False), 
                                        filter_poza_name,
                            pd.np.where(df_fixed["name"].str.contains('|'.join(filter_sadik), case = False), 
                                        filter_sadik_name,
                            pd.np.where(df_fixed["name"].str.contains('|'.join(filter_shkola), case = False), 
                                        filter_shkola_name, 
                                        filter_inshe)))))))
    
    df_fixed = df_fixed[(df_fixed['type'] != "Інше") & (df_fixed['type'] != "Позашкільне")]
    
    df_fix_temp_num = df_fixed.copy()
    df_fix_temp_num['name_fixed_num'] = df_fix_temp_num['name'].str.extract(reg_number, expand = False)
    

    df_fix_temp_str = df_fix_temp_num[df_fix_temp_num['name_fixed_num'].isnull()]
    
    df_fix_temp_num = df_fix_temp_num[df_fix_temp_num['name_fixed_num'].notnull()]    
    df_fix_temp_num['name_fixed_num'] = df_fix_temp_num['name_fixed_num'][df_fix_temp_num['name_fixed_num'].
                                  str.contains('№', case = False, na = False)].str.replace('\s','')

    df_fix_temp_str['name_fixed_num'] = df_fix_temp_str['name'].str.extract(reg_quotes, expand = False)
    df_fix_temp_str = df_fix_temp_str[df_fix_temp_str['name_fixed_num'].notnull()]
    
    frames = [df_fix_temp_num, df_fix_temp_str]
    df_reborn = pd.concat(frames)
    
    
    """
    ВИТЯГУВАННЯ МІСТ З АДРЕС
    """
    df_reborn['full'] = df_reborn['full'].str.replace('м. ','м.')
    df_reborn['city_new'] = df_reborn['full'].str.extract(reg_city, expand = False)
    df_reborn['city_new'] = df_reborn['city_new'].str.replace('м\.','')
    df_reborn['city_new'] = df_reborn['city_new'].str.replace('Івано','Івано-Франківськ')
    
    print("Було закладів: ", len(df_reborn))
    df_reborn = df_reborn[df_reborn.city_new.str.contains('|'.join(regions), na = False, case=False)]   
    
    
    """
    СТВОРЕННЯ КОЛОНКИ З МІСТОМ І ШКОЛОЮ
    """ 
    df_reborn['city_fixed'] = df_reborn['city_new'] + " " + df_reborn['type'] + " " + df_reborn['name_fixed_num']
    
    df_reborn = df_reborn.replace(np.nan, '', regex=True)
    
    df_reborn['street'] = df_reborn['street'].astype(str)
    df_reborn['street2'] = df_reborn['street2'].astype(str)

    
    """
    СТВОРЕННЯ КОЛОНКИ З АЛЬТЕРНАТИВНОЮ АДРЕСОЮ З МІСТОМ І ВУЛИЦЕЮ
    """
    df_reborn['address_fixed'] = df_reborn['city_new'] + " " + df_reborn['street'] + " " + df_reborn['street2']
    
    df_reborn['address_fixed'] = df_reborn['address_fixed'].str.replace("  "," ")
    
    
    """
    ПІДРАХУНОК КІЛЬКОСТІ СЛІВ У КОЛОНЦІ З НАЗВОЮ ШКІЛ. 
    ЯКЩО БІЛЬШЕ 1, ТО ШВИДШЕ ЗА ВСЕ, ЦЕ НЕ НАЗВА ЗАКЛАДУ
    """
    df_reborn['word_count'] = df_reborn['name_fixed_num'].str.split().str.len()
    df_reborn = df_reborn[(df_reborn.word_count == 1) | (df_reborn.name_fixed_num.str.contains('ім\.'))]
    
    
    """
    ВИДАЛЕННЯ ЗАЙВИХ МІСТ
    """
    bad_cities = ['Дніпрорудне','Верхньодніпровськ', 'Миколаївка', 'Новомиколаївка', 'Верхньодніпровськ', 'Чернігівка', 'Дніпровське']
    df_reborn = df_reborn[~df_reborn['city_fixed'].str.contains('|'.join(bad_cities), case = False)]
    print("Залишилось після фільтрування:", len(df_reborn)) 
    print("Міста: ", df_reborn.city_new.unique())
    
#     df_reborn = df_reborn[['city_fixed', 'address_fixed', 'full_2', 'full_3']]
    
    return df_reborn, df_fix_temp_str

In [13]:
df_reborn, df_fix_temp_str = fix_table()

Було закладів:  19067
Залишилось після фільтрування: 2174
Міста:  ['Дніпро' 'Івано-Франківськ' 'Запоріжжя' 'Черкаси' 'Харків' 'Чернівці'
 'Херсон' 'Миколаїв' 'Вінниця' 'Хмельницький' 'Львів' 'Кропивницький'
 'Суми' 'Одеса' 'Житомир' 'Рівне' 'Луцьк' 'Тернопіль' 'Ужгород' 'Полтава'
 'Чернігів' 'Київ']


In [32]:
names_in_quotes = df_fix_temp_str.name_fixed_num.unique().tolist()
names_in_quotes

['Гармонія',
 'Канівська загальноосвітня санаторна школа-інтернат І-ІІІ ступенів Черкаської обласної ради',
 'Новоолександрівський навчально-виховний комплекс ',
 'ПРОМЕТЕЙ',
 'Азимут',
 'Загальноосвітній начальний заклад І-ІІІ ступенів -дошкільний навчальний заклад',
 'Шевченківський навчально-виховний комплекс ',
 'загальноосвітній навчальний заклад і-ііі ступенів - дошкільний навчальний заклад',
 'Загальноосвітній навчальний заклад І-ІІІ ступенів - дошкільний навчальний заклад',
 'Загальноосвітній навчальний заклад І-ІІІ ступенів - дошкільній навчальний заклад ім. Ф. Конюхова',
 'Гірсівська загальноосвітня школа І - ІІІ ступенів Гірсівської сільської ради Запорізької області',
 'Всесвіт',
 'Дніпровська загальноосвітня школа І-ІІІ ступенів',
 'Світоч',
 'Запорізький обласний ліцей-інтернат з посиленою військово-фізичною підготовкою ',
 'Преславська спеціальна загальноосвітня школа-інтернат',
 'Чернігівська загальноосвітня школа-інтернат І-ІІІ ступенів',
 'Запорізька спеціалізована шк

In [210]:
filter_keywords = ['школа', 'шкіль', 'загальноосвітн', 
                   'гімназ', 'ліце', 'колегіум',
                   'садок', 'дошкільн', 'днз', 'ясла', 'ясел', 'дюсш', 'нвк']

df_fix_temp_str_t = df_fix_temp_str.copy()

df_fix_temp_str_t['name_fixed_num'] = df_fix_temp_str_t['name_fixed_num'].str.lstrip()#прибирає пробіл на початку
df_fix_temp_str_t['name_fixed_num'] = df_fix_temp_str_t['name_fixed_num'].str.rstrip()#прибирає пробіл в кінці
count = df_fix_temp_str_t['name_fixed_num'].str.count('\s')
dropped = df_fix_temp_str_t[(count == 0)].copy()
dropped = dropped['name_fixed_num'].unique().tolist()

In [ ]:
""" З УСІХ ВЛАСНИХ НАЗВ names_in_quotes ЗАЛИШАЮТЬСЯ ТІЛЬКИ ТІ, ДЕ Є 1 СЛОВО, ЯКЕ БІЛЬШЕ НІ 4 СИМВОЛИ """

short = [s for s in dropped if len(s) >= 4]
df_short = pd.DataFrame(short)
df_short = df_short[~df_short[0].str.contains('|'.join(filter_keywords), case = False)]
df_short.to_csv('school_names.csv', index = True, header = True)

In [211]:
pd.read_csv('school_names.csv', index_col = 0)

,0
0,Гармонія
1,ПРОМЕТЕЙ
2,Азимут
3,Всесвіт
4,Світоч
5,Джерело
6,Ерудит
7,Молодь
8,Перспектива
9,Орієнтир


In [335]:
df_fix_temp_str.to_csv('minosvity_fixed.csv', index = True, header = True)